# 네이버 쇼핑리뷰 감성분석
- GRU
- Mecab 형태소 분석기

### Mecab 설치

In [1]:
# !rm -rf Mecab-ko-for-Google-Colab

In [2]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 3.84 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 66.5 MB/s 
     |████████████████████████████████| 453 kB 43.9 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-05-27 05:07:34--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (b

### 데이터 탐색

In [3]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Mecab
mecab = Mecab()

In [11]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'
df = pd.read_table(url, names=['ratings','reviews'])
df.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [12]:
# 평점이 4, 5점인 데이터를 1(긍정), 나머지는 0(부정)
df['label'] = df.ratings.apply(lambda x: 1 if x >= 4 else 0)
df.head()

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


### 데이터 전처리

In [13]:
print(df.shape)
# Null data 확인
print('결측치:',df.isna().sum().sum())
# 중복데이터 확인
print('유일값:',df.reviews.nunique())
# 중복 제거 
df. drop_duplicates(subset=['reviews'], inplace=True)
df.shape

(200000, 3)
결측치: 0
유일값: 199908


(199908, 3)

- 한글 이외의 데이터는 제거

In [14]:
df.reviews = df.reviews.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [16]:
# Null 데이터가 생기면 제거
df.reviews.replace('', np.nan, inplace=True)
df.isna().sum().sum()

0

- 훈련/테스트 데이터셋 분리

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.reviews, df.label.values, stratify=df.label.values,
    test_size=0.2, random_state=2022
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((159926,), (39982,), (159926,), (39982,))

### 토큰화

In [18]:
from konlpy.tag import Mecab
mecab = Mecab()

In [19]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']

In [21]:
from tqdm.notebook import tqdm

train_data = []
for sentence in tqdm(X_train):
    morphs = mecab.morphs(sentence)
    tmp_X = [word for word in morphs if word not in stopwords]
    train_data.append(tmp_X)

  0%|          | 0/159926 [00:00<?, ?it/s]

In [22]:
test_data = []
for sentence in tqdm(X_test):
    morphs = mecab.morphs(sentence)
    tmp_X = [word for word in morphs if word not in stopwords]
    test_data.append(tmp_X)

  0%|          | 0/39982 [00:00<?, ?it/s]

In [24]:
train_data[0]

['재', '구매', '늘', '먹', '던', '거', '예요', '밥맛', '좋', '아요']

- 인코딩

In [31]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(train_data)

In [33]:
len(t.word_index)

41261

In [34]:
# 등장 빈도가 3 미만인 것의 개수
threshold = 3
total_cnt = len(t.word_index)   # 308969
rare_cnt = 0        # 등장 빈도가 threshold 보다 작은 단어의 개수
total_freq = 0       # 훈련 데이터의 전체 단어 빈도수의 합
rare_freq = 0       # 등장 빈도가 threshold 보다 작은 단어의 등장 빈도수의 합

In [35]:
for key, value in t.word_counts.items():
    total_freq += value
    if value < threshold:
        rare_cnt += 1
        rare_freq += 1

In [37]:
print('단어 집합(vocabulary)의 크기 :', total_cnt)
print(f'등장 빈도가 {threshold - 1}번 이하인 희귀 단어의 수: {rare_cnt}')
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 41261
등장 빈도가 2번 이하인 희귀 단어의 수: 23747
단어 집합에서 희귀 단어의 비율: 57.553137345192795
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.9081060327655455


- 등장 빈도가 2회 이하인 단어는 제외하고 토큰화

In [38]:
# 0번 패딩 토큰, 1번 OOV(Out-Of-Value) 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
vocab_size

17516

In [39]:
t = Tokenizer(num_words=vocab_size, oov_token='OOV')
t.fit_on_texts(train_data)
X_train = t.texts_to_sequences(train_data)
X_test = t.texts_to_sequences(test_data)

In [40]:
# 데이터의 최대/평균 길이
max(len(s) for s in X_train), sum(map(len, X_train)) / len(X_train)

(86, 16.351331240698823)

In [41]:
# 리뷰 길이를 60으로 설정하고 패딩
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 60
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((159926, 60), (39982, 60))

### GRU 모델
- 임베딩 벡터의 개수: 100
- GRU 유닛의 개수: 128

In [43]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [44]:
model = Sequential([
    Embedding(vocab_size, 100, input_length=max_len),
    GRU(128),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 100)           1751600   
                                                                 
 gru (GRU)                   (None, 128)               88320     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,840,049
Trainable params: 1,840,049
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.compile('adam', 'binary_crossentropy', ['accuracy'])
model_path = 'best-shopping-gru.h5'
mc2 = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=3)

In [46]:
hist = model.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=128, callbacks=[mc2,es2]
)
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

Epoch 1/30
1000/1000 [==============================] - ETA: 0s - loss: 0.2850 - accuracy: 0.8891
Epoch 1: val_loss improved from inf to 0.23794, saving model to best-shopping-gru.h5
1000/1000 [==============================] - 209s 205ms/step - loss: 0.2850 - accuracy: 0.8891 - val_loss: 0.2379 - val_accuracy: 0.9125
Epoch 2/30
1000/1000 [==============================] - ETA: 0s - loss: 0.2084 - accuracy: 0.9257
Epoch 2: val_loss improved from 0.23794 to 0.22693, saving model to best-shopping-gru.h5
1000/1000 [==============================] - 203s 203ms/step - loss: 0.2084 - accuracy: 0.9257 - val_loss: 0.2269 - val_accuracy: 0.9163
Epoch 3/30
1000/1000 [==============================] - ETA: 0s - loss: 0.1763 - accuracy: 0.9387
Epoch 3: val_loss did not improve from 0.22693
1000/1000 [==============================] - 195s 195ms/step - loss: 0.1763 - accuracy: 0.9387 - val_loss: 0.2391 - val_accuracy: 0.9128
Epoch 4/30
1000/1000 [==============================] - ETA: 0s - loss: 0.

[0.23377080261707306, 0.9139862656593323]

### 실제 데이터 예측

In [47]:
def sentiment_predict(review, tokenizer=t, max_len=max_len):
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',review).strip()
    morphs = mecab.morphs(review)
    morphs = [word for word in morphs if word not in stopwords]
    encoded = tokenizer.texts_to_sequences([morphs])
    padded = pad_sequences(encoded, maxlen=max_len)
    score = float(best_model.predict(padded))
    return f'긍정({score*100:.2f}%)' if score > 0.5 else f'부정({(1-score)*100:.2f}%)'

In [48]:
sentiment_predict('이 상품 진짜 좋아요... 저는 강추합니다. 대박')

'긍정(94.32%)'